# Numeric TIC TAC TOE

In [1]:
import numpy as np
from enum import Enum

In [2]:
p1_pawns = np.array([v for v in range(1,10,2)])
p2_pawns = np.array([v for v in range(2,10,2)])

In [3]:
def get_board(size=(3,3)):
    return np.zeros(size, dtype='uint8')
    
get_board()


array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [162]:
class Minimax():
    def __init__(self, max_depth):
        self.max_depth = max_depth        
    
    def get_possible_actions(self, grid, pawns):
        result = []
        availables_moves = np.array(np.where(grid == 0)).T
                
        for i,j in availables_moves:
            for k in pawns:                
                result.append(tuple((i,j,k)))
        
        return result

class State():
    class Value(Enum):
        WIN=1
        DRAW=0
        NONE=-1
    
    def __init__(self, win_score=15):        
        self.win_score = 15        
        self.value = self.Value.NONE                
        
    def evaluate(self, grid):        
        # 1 for a good situation
        # -1 for an opponent given victory
        # 0.5 for a draw situation
        # 0 for a move with low consequences
                
        # win on a row
        if self.win_score in np.sum(grid, axis=1):
            return 1 
            
        # win on a col
        if self.win_score in np.sum(grid, axis=0):
            return 1
        
        # win on the diag
        if self.win_score == np.diag(grid).sum():
            return 1
        
        # win on the inverse diag
        if self.win_score == np.diag(np.fliplr(grid)).sum():
            return 1                
        
        # draw if all is filled and nobody has win
        if len(np.where(grid == 0)[0]) == 0:
            return 0.5
                        
        return 0

    def is_terminal(self, grid):
        return self.evaluate(grid, []) in [0,1]
    
    
class Board():
    def __init__(self,size=(3,3)):
        self.grid = np.zeros(size, dtype='uint8')        
        
    def update(self, move):
        if self.grid[move[0], move[1]] == 0:
            self.grid[move[0], move[1]] = move[2]
            return True
        else:
            return False
        
    
    def simulate(self, move):
        sgrid = self.grid.copy()
        sgrid[move[0], move[1]] = move[2]
        return sgrid

p1_pawns = [3,5,7,9]
p2_pawns = [4,6,8]
# mm = Minimax(2)

board = Board()

board.update((0,0,1))
board.update((1,1,2))
# board.update((2,2,3))
# board.update((0,1,6))
# board.update((1,2,5))
# board.update((0,2,4))
    
print(board.grid)
state = State()

# for action in mm1.get_possible_actions(board.grid, p1_pawns):
#     print(action, state.evaluate(board.simulate(action)))

def get_possible_actions(grid, pawns):
        result = []
        availables_moves = np.array(np.where(grid == 0)).T
                
        for i,j in availables_moves:
            for k in pawns:                
                result.append(tuple((i,j,k,0,"0")))
        
        return result
    
def recursive_exploration(grid, pfirst_pawns, psecond_pawns, action=(0,0,0,0,"0"), factor=-1, actions = [], limit=3):
    # print("recursive_exploration: ", pfirst_pawns, psecond_pawns)
    value = state.evaluate(grid)        
    
    # not ended game
    if value == 0 and len(action[4].split(".")) < limit:                
        actions.append(tuple(action))
        branch = action[4]
        k = 0
        # evaluate all possible actions
        for action in get_possible_actions(grid, pfirst_pawns):
            # increment the branch
            k += 1
            # remove the pawn used in the action
            first_pawns = [pawn for pawn in pfirst_pawns if pawn != action[2]]
            # update the grid
            new_grid = grid.copy()
            new_grid[action[0], action[1]] = action[2]            
            # continue exploration                        
            recursive_exploration(new_grid, psecond_pawns, first_pawns, (action[0], action[1], action[2], value*factor, branch + "." + str(k)), factor*-1, actions, limit)
    # no more places to play
    else:                
        # actions.append(tuple(action))
        actions.append(tuple((action[0], action[1], action[2], value*factor, action[4])))
    
    return actions

possibilities = recursive_exploration(board.grid, p2_pawns, p1_pawns, limit=4)




[[1 0 0]
 [0 2 0]
 [0 0 0]]


In [142]:
possibilities[:10]

[(0, 0, 0, 0, '0'),
 (0, 1, 4, 0, '0.1'),
 (0, 2, 3, 0, '0.1.1'),
 (1, 0, 6, 0, '0.1.1.1'),
 (1, 0, 8, 0, '0.1.1.2'),
 (1, 2, 6, 0, '0.1.1.3'),
 (1, 2, 8, 0, '0.1.1.4'),
 (2, 0, 6, 0, '0.1.1.5'),
 (2, 0, 8, 0, '0.1.1.6'),
 (2, 1, 6, 0, '0.1.1.7')]

In [148]:
def get_max_score_node(possibilities):
    # search for the max score 
    max_score = np.array([node[3] for node in possibilities]).max()
    
    path = []
    # search for the node with max score
    for node in possibilities:
        if node[3] == max_score:
            path.append(node)
            break;
    
    parents_paths = path[0][4].split(".")    
    
    # search for the complete path toward the node     
    for i in range(0,len(parents_paths)-1):
        parent_path = ".".join(parents_paths[:i+1])
        path.append([node for node in possibilities if node[4] == parent_path])
    
    return path
    
path = get_max_score_node(possibilities)

In [166]:
def game():
    board = Board()
    quit = False
    turn = 0
    pawns = [[1,3,5,7,9], [2,4,6,8]]    
    
    while state.evaluate(board.grid) == 0 and not quit:
        turn += 1
        print("turn {}".format(turn))
        print("P1:", pawns[0], "\tP2: ", pawns[1])        
        
        print(board.grid)
        
        while not move_validate:
            option = input("[Q] Quit [row,col,pawn] To play\n")
                    
            quit = option == "Q":
            if quit:
                break;
        
            move_validate = False
        
                option = input("[Q] Quit [row,col,pawn] To play\n")
                move = [int(value) for value in option.split(",")]            
                if move[2] in pawns[turn % 2]:
                    move_validate = board.update(move)            
                    print(move_validate)
                    if not move_validate:
                        print("Invalid move")
                else:
                    print("invalid pawn")
        
        if quit:
                break;
                
    print("Thanks for playing")

game()

turn 1
P1: [1, 3, 5, 7, 9] 	P2:  [2, 4, 6, 8]
[[0 0 0]
 [0 0 0]
 [0 0 0]]


[Q] Quit [row,col,pawn] To play
 1,1,1
[Q] Quit [row,col,pawn] To play
 q


ValueError: invalid literal for int() with base 10: 'q'

In [99]:
class Minimax():
    def __init__(self, max_depth):
        self.max_depth = max_depth        
    
    def get_possible_actions(self, grid, pawns):
        result = []
        availables_moves = np.array(np.where(grid == 0)).T
                
        for i,j in availables_moves:
            for k in pawns:                
                result.append(tuple((i,j,k)))
        
        return result
    
        
        
            
        


class State():
    class Value(Enum):
        WIN=1
        DRAW=0
        NONE=-1
    
    def __init__(self, win_score=15):        
        self.win_score = 15        
        self.value = self.Value.NONE                
        
    def evaluate(self, grid):        
        # 1 for a good situation
        # -1 for an opponent given victory
        # 0.5 for a draw situation
        # 0 for a move with low consequences
                
        # win on a row
        if self.win_score in np.sum(grid, axis=1):
            return 1 
            
        # win on a col
        if self.win_score in np.sum(grid, axis=0):
            return 1
        
        # win on the diag
        if self.win_score == np.diag(grid).sum():
            return 1
        
        # win on the inverse diag
        if self.win_score == np.diag(np.fliplr(grid)).sum():
            return 1                
        
        # draw if all is filled and nobody has win
        if len(np.where(grid == 0)[0]) == 0:
            return 0.5
                        
        return 0

    def is_terminal(self, grid):
        return self.evaluate(grid, []) in [0,1]
    
    
class Board():
    def __init__(self,size=(3,3)):
        self.grid = np.zeros(size, dtype='uint8')        
        
    def update(self, move):
        self.grid[move[0], move[1]] = move[2]
    
    def simulate(self, move):
        sgrid = self.grid.copy()
        sgrid[move[0], move[1]] = move[2]
        return sgrid

p1_pawns = [7,9]
p2_pawns = [8]
    
mm = Minimax(2)

board = Board()

board.update((0,0,1))
board.update((1,1,2))
board.update((2,2,3))
board.update((0,1,6))
board.update((1,2,5))
board.update((0,2,4))

print(board.grid)
state = State()

# for action in mm1.get_possible_actions(board.grid, p1_pawns):
#     print(action, state.evaluate(board.simulate(action)))

def get_possible_actions(grid, pawns):
        result = []
        availables_moves = np.array(np.where(grid == 0)).T
                
        for i,j in availables_moves:
            for k in pawns:                
                result.append(tuple((i,j,k)))
        
        return result
    
def recursive_exploration(grid, pfirst_pawns, psecond_pawns):
    value = state.evaluate(grid)
    
    actions = np.array([])
    
    # not ended game
    if value == 0:        
        # evaluate all childs
        for action in get_possible_actions(grid, pfirst_pawns):
            print("action: ", action)
            first_pawns = [pawn for pawn in pfirst_pawns if pawn != action[2]]
            print("first_pawns: ", first_pawns)
            
            grid[action[0], action[1]] = action[2]
            print("grid: ", grid)
            actions += recursive_exploration(board.simulate(action), psecond_pawns, first_pawns)
            
    else:        
        print("returns :", actions)
        return actions

print(recursive_exploration(board.grid, p1_pawns, p2_pawns))

[[1 6 4]
 [0 2 5]
 [0 0 3]]
action:  (1, 0, 7)
first_pawns:  [9]
grid:  [[1 6 4]
 [7 2 5]
 [0 0 3]]
action:  (2, 0, 8)
first_pawns:  []
grid:  [[1 6 4]
 [7 2 5]
 [8 0 3]]
action:  (2, 1, 9)
first_pawns:  []
grid:  [[1 6 4]
 [7 2 5]
 [8 9 3]]


TypeError: ufunc 'add' output (typecode 'O') could not be coerced to provided output parameter (typecode 'd') according to the casting rule ''same_kind''

In [ ]:
avm = np.array([[0, 0, 0, 1, 1, 1, 2, 2, 2],[0, 1, 2, 0, 1, 2, 0, 1, 2]])

list(avm.T)
#for i, j in enumerate(avm):
#    print(i,j)



In [ ]:
def try_move(grid, pawns):
    # search for availables moves 
    return np.where(grid == 0))
                     

try_move(get_board(), p2_pawns)